In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
EMA + RSI + MACD + Bollinger Swing Strategy
-------------------------------------------
Framework: cap 25% deploy/day, VOLAR ranking, MVO sizing, optional 52w proximity, fees, exports & equity plot.

ENTRY (signal day t, fill on t+1 open by default):
  • Up-trend: EMA_fast > EMA_slow  (and optional HTF gate: Close > EMA200)
  • RSI oversold cross-up
  • MACD bullish crossover
  • Bollinger reversion (below lower recently, back above lower today)

  To avoid ultra-rare "all on the same bar" alignment, you can:
    - set require_same_day=False (default), AND
    - require min_confirmations of the above {RSI, MACD, BB} within short rolling windows.

EXIT (any):
  • RSI > overbought
  • MACD bearish cross
  • Upper-band reversal: (Close_{t-1} ≥ Upper_{t-1}) & (Close_t < Upper_t)
  • Intraday SL/TP using next-day High/Low; if both hit same day, SL wins (conservative)

You can tune:
  - Confirmation windows (bars): rsi_confirm_window / macd_confirm_window / bb_confirm_window
  - N-of-3 confirmations (min_confirmations)
  - require_same_day=True to force strict same-bar confirmations (will reduce signals)
  - enforce_52w_filter to require price ≥ X% of 52w-high at entry day
"""

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
    import matplotlib.pyplot as plt
except Exception:
    # If your environment lacks these, install them: pip install yfinance matplotlib
    pass

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("ema_rsi_macd_bb_cap25_nof3")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2010-01-01"
    end_date: str   = "2025-10-01"
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = True

    # EMA Trend
    ema_fast: int   = 10
    ema_slow: int   = 50
    ema_htf:  int   = 200             # higher timeframe EMA (gate)
    use_htf_trend: bool = True        # require Close > EMA200

    # RSI
    rsi_len: int    = 14
    rsi_os: float   = 35.0            # a bit looser than 30 to generate more signals
    rsi_ob: float   = 70.0

    # MACD
    macd_fast: int  = 12
    macd_slow: int  = 26
    macd_signal: int = 9

    # Bollinger
    bb_len: int     = 20
    bb_std: float   = 2.0

    # ADX (optional; kept for logs/analytics)
    adx_len: int = 14
    adx_min: float = 20.0

    # Intraday SL/TP (SL wins ties)
    stop_loss_pct: float = 0.05
    target_pct: float    = 0.10

    # Portfolio
    apply_fees: bool    = True
    initial_capital: float = 200_000.0
    max_concurrent_positions: int = 4
    deploy_cash_frac: float = 0.25
    top_k_daily: int = 3

    # Execution (fills next-day open by default)
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Ranking & filters
    benchmark_try: Tuple[str,...] = ("^CNX500","^CRSLDX","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.70
    enforce_52w_filter: bool = False   # default OFF to avoid choking signals; set True if you prefer

    # Liquidity guard (optional)
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

    # ==== ENTRY FLEXIBILITY ====
    # If True: RSI+MACD+BB must ALL occur the same day (strict; low trade count)
    # If False: require min_confirmations within the rolling windows below
    require_same_day: bool = False

    # Confirmation windows (bars)
    rsi_confirm_window: int = 5
    macd_confirm_window: int = 5
    bb_confirm_window: int = 3

    # How many of {RSI, MACD, BB} must be true when require_same_day=False
    min_confirmations: int = 2

CFG = Config()

# =========================
# FEES (per your earlier framework)
# =========================
APPLY_FEES = True

def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    """Approx brokerage + taxes (Groww-like caps) – same as your framework."""
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0:
            return 0.0
        fee = turnover * BROKER_PCT
        fee = max(BROKER_MIN, min(fee, BROKER_CAP))
        return fee

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# =========================
# Helpers
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    """Load universe from list or file; enforce .NS suffix and uniqueness."""
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError(
            "Provide CFG.static_symbols=[...] ('.NS' suffixes) or set CFG.static_symbols_path "
            "to a file containing one symbol per line."
        )
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    uniq = []
    seen = set()
    for s in out:
        if s not in seen:
            uniq.append(s)
            seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    """Fetch OHLCV from yfinance with simple parquet cache; auto-adjusted prices."""
    ensure_dirs(cache_dir)
    data = {}
    end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df
                    continue
            except Exception:
                pass
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)  # Open, High, Low, Close, Volume
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data

# =========================
# Indicators
# =========================
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0.0)).rolling(length).mean()
    loss = (-delta.where(delta < 0, 0.0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50.0)

def macd(close: pd.Series, fast: int=12, slow: int=26, signal: int=9):
    ema_fast = close.ewm(span=fast, adjust=False, min_periods=fast).mean()
    ema_slow = close.ewm(span=slow, adjust=False, min_periods=slow).mean()
    line = ema_fast - ema_slow
    sig  = line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    hist = line - sig
    return line, sig, hist

def bollinger(close: pd.Series, length: int=20, nstd: float=2.0):
    ma = close.rolling(length).mean()
    sd = close.rolling(length).std(ddof=0)
    upper = ma + nstd*sd
    lower = ma - nstd*sd
    return ma, upper, lower

def _true_range(high: pd.Series, low: pd.Series, prev_close: pd.Series) -> pd.Series:
    return pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

def adx(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14) -> pd.Series:
    prev_high = high.shift(1)
    prev_low  = low.shift(1)
    prev_close = close.shift(1)

    up_move   = high - prev_high
    down_move = prev_low - low
    plus_dm  = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)

    tr = _true_range(high, low, prev_close)

    alpha = 1.0 / length
    atr = tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)

    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx_series = dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    return adx_series

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    """Compute all indicators; drop rows with NaNs from initial warmups."""
    out = df.copy()
    # EMAs
    out["ema_fast"] = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"] = ema(out["Close"], cfg.ema_slow)
    out["ema_htf"]  = ema(out["Close"], cfg.ema_htf)
    # RSI
    out["rsi"]      = rsi(out["Close"], cfg.rsi_len)
    out["rsi_prev"] = out["rsi"].shift(1)
    # MACD
    macd_line, macd_sig, macd_hist = macd(out["Close"], cfg.macd_fast, cfg.macd_slow, cfg.macd_signal)
    out["macd"] = macd_line
    out["macd_sig"] = macd_sig
    out["macd_prev"] = out["macd"].shift(1)
    out["macd_sig_prev"] = out["macd_sig"].shift(1)
    # Bollinger
    bb_ma, bb_up, bb_lo = bollinger(out["Close"], cfg.bb_len, cfg.bb_std)
    out["bb_mid"] = bb_ma
    out["bb_up"]  = bb_up
    out["bb_lo"]  = bb_lo
    out["close_prev"] = out["Close"].shift(1)
    out["bb_up_prev"] = out["bb_up"].shift(1)
    out["bb_lo_prev"] = out["bb_lo"].shift(1)
    # ADX (optional)
    out["adx"]      = adx(out["High"], out["Low"], out["Close"], cfg.adx_len)
    # Extras
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["trend_strength"] = (out["ema_fast"] - out["ema_slow"]) / out["Close"]
    out["high_52w"] = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity:
        return True
    if row["Close"] < cfg.min_price_inr:
        return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d:
        return False
    return True

# =========================
# Strategy simulator
# =========================
def simulate_ticker(ticker: str, df: pd.DataFrame, cfg: Config):
    """
    Generate raw BUY/SELL legs with your entry/exit logic.
    BUY legs are 'candidates'; portfolio engine sizes & filters them later.
    """
    d = compute_indicators(df, cfg).copy()
    cols = ["ticker","side","date","price","shares","reason","signal_reason","score",
            "rsi","macd","macd_sig","bb_up","bb_lo","ema_fast","ema_slow","ema_htf",
            "trend_strength","close","high_52w","adx"]
    if d.empty:
        return pd.DataFrame(columns=cols), pd.Series(dtype=float)

    # ---- Entry confirmations ----
    trend_ok = (d["ema_fast"] > d["ema_slow"])
    htf_ok   = (d["Close"] > d["ema_htf"]) if cfg.use_htf_trend else pd.Series(True, index=d.index)

    # primitive "events"
    rsi_cross_up = (d["rsi"].shift(1) < cfg.rsi_os) & (d["rsi"] >= cfg.rsi_os)
    macd_bull    = (d["macd"].shift(1) <= d["macd_sig"].shift(1)) & (d["macd"] > d["macd_sig"])
    # BB reversion concept: recently below lower, today back above lower
    was_below_lo_recent = ( (d["Close"] < d["bb_lo"]) | (d["Close"].shift(1) < d["bb_lo"].shift(1)) )
    bb_back_above_today = (d["Close"] > d["bb_lo"])
    bb_reversion_same   = (d["Close"].shift(1) < d["bb_lo"].shift(1)) & (d["Close"] > d["bb_lo"])

    if cfg.require_same_day:
        confirm_rsi  = rsi_cross_up
        confirm_macd = macd_bull
        confirm_bb   = bb_reversion_same
        need_confirms = 3
    else:
        confirm_rsi  = rsi_cross_up.rolling(cfg.rsi_confirm_window).max().astype(bool)
        confirm_macd = macd_bull.rolling(cfg.macd_confirm_window).max().astype(bool)
        below_recent = was_below_lo_recent.rolling(cfg.bb_confirm_window).max().astype(bool)
        # require the "below lower" to have occurred recently (before today), and today we closed back above lower
        confirm_bb   = below_recent.shift(1).fillna(False) & bb_back_above_today
        need_confirms = cfg.min_confirmations

    confirmations = confirm_rsi.astype(int) + confirm_macd.astype(int) + confirm_bb.astype(int)
    entry_signal  = trend_ok & htf_ok & (confirmations >= need_confirms)

    # ---- Exit indicators ----
    rsi_exit   = (d["rsi"] > cfg.rsi_ob)
    macd_bear  = (d["macd"].shift(1) >= d["macd_sig"].shift(1)) & (d["macd"] < d["macd_sig"])
    upper_revr = (d["Close"].shift(1) >= d["bb_up"].shift(1)) & (d["Close"] < d["bb_up"])
    indicator_exit = rsi_exit | macd_bear | upper_revr

    in_pos = False
    entry_px = stop_px = tgt_px = 0.0
    trades = []

    idx = list(d.index)
    for i in range(len(idx)-1):
        dt, nxt = idx[i], idx[i+1]
        row, nxt_row = d.loc[dt], d.loc[nxt]

        if not in_pos:
            if entry_signal.loc[dt] and basic_liquidity_ok(row, cfg):
                px = float(nxt_row["Open"] if cfg.entry_on_next_open else row["Close"])
                sig_reason = (
                    f"EMA{cfg.ema_fast}>{cfg.ema_slow}"
                    f"{' & Close>EMA200' if cfg.use_htf_trend else ''}; "
                    f"Confirms: RSI({cfg.rsi_confirm_window}), MACD({cfg.macd_confirm_window}), "
                    f"BB({cfg.bb_confirm_window}) need {need_confirms}"
                )
                trades.append({
                    "ticker": ticker, "side": "BUY", "date": nxt if cfg.entry_on_next_open else dt,
                    "price": px, "shares": 0, "reason": "candidate",
                    "signal_reason": sig_reason,
                    # score: trend strength + MACD distance (simple heuristic)
                    "score": float(1000.0*((row["ema_fast"]-row["ema_slow"])/row["Close"])) \
                             + float(row["macd"] - row["macd_sig"]),
                    "rsi": float(row["rsi"]),
                    "macd": float(row["macd"]), "macd_sig": float(row["macd_sig"]),
                    "bb_up": float(row["bb_up"]), "bb_lo": float(row["bb_lo"]),
                    "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]), "ema_htf": float(row["ema_htf"]),
                    "trend_strength": float(row["trend_strength"]), "close": float(row["Close"]),
                    "high_52w": float(row["high_52w"]), "adx": float(row["adx"])
                })
                in_pos = True
                entry_px = px
                stop_px = entry_px * (1 - cfg.stop_loss_pct)
                tgt_px  = entry_px * (1 + cfg.target_pct)

        else:
            # Intraday SL/TP using next day's H/L (SL wins ties)
            exec_date = nxt if cfg.exit_on_next_open else dt
            hit = None
            nxt_low, nxt_high = float(nxt_row["Low"]), float(nxt_row["High"])

            if (nxt_low <= stop_px) and (nxt_high >= tgt_px):
                hit, exec_price = "stop", float(stop_px)      # SL priority
            elif nxt_low <= stop_px:
                hit, exec_price = "stop", float(stop_px)
            elif nxt_high >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            elif indicator_exit.loc[dt]:
                exec_price = float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
                if rsi_exit.loc[dt]:
                    hit = "indicator_exit_rsi_ob"
                elif macd_bear.loc[dt]:
                    hit = "indicator_exit_macd_bear"
                elif upper_revr.loc[dt]:
                    hit = "indicator_exit_bb_upper_reversal"
                else:
                    hit = "indicator_exit"

            if hit is not None:
                trades.append({
                    "ticker": ticker, "side": "SELL", "date": exec_date,
                    "price": float(exec_price), "shares": 0, "reason": hit,
                    "signal_reason": "", "score": np.nan,
                    "rsi": float(row["rsi"]),
                    "macd": float(row["macd"]), "macd_sig": float(row["macd_sig"]),
                    "bb_up": float(row["bb_up"]), "bb_lo": float(row["bb_lo"]),
                    "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]), "ema_htf": float(row["ema_htf"]),
                    "trend_strength": float(row["trend_strength"]), "close": float(row["Close"]),
                    "high_52w": float(row["high_52w"]), "adx": float(row["adx"])
                })
                in_pos = False
                entry_px = stop_px = tgt_px = 0.0

    # Force-close if still in position at the end
    if in_pos:
        last_dt = d.index[-1]; row = d.loc[last_dt]
        trades.append({
            "ticker": ticker, "side": "SELL", "date": last_dt,
            "price": float(row["Close"]), "shares": 0, "reason": "final_close",
            "signal_reason": "", "score": np.nan,
            "rsi": float(row["rsi"]),
            "macd": float(row["macd"]), "macd_sig": float(row["macd_sig"]),
            "bb_up": float(row["bb_up"]), "bb_lo": float(row["bb_lo"]),
            "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]), "ema_htf": float(row["ema_htf"]),
            "trend_strength": float(row["trend_strength"]), "close": float(row["Close"]),
            "high_52w": float(row["high_52w"]), "adx": float(row["adx"])
        })

    return pd.DataFrame(trades, columns=cols), pd.Series(dtype=float)

# =========================
# Ranking / Portfolio
# =========================
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t)
            return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty:
            scores[t] = 0.0
            continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty:
                scores[t] = 0.0
                continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)):
            scores[t] = 0.0
            continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    """Simple long-only mean-variance with non-negativity via active-set pruning."""
    n = len(mu)
    eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)

    def solve_lambda(lmbd: float, active_mask=None):
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b)
                w = sol[:n]
            except np.linalg.LinAlgError:
                w = np.full(n, 1.0/n)
            return w
        else:
            idx = np.where(active_mask)[0]
            if len(idx)==0:
                return np.full(n, 1.0/n)
            S = Sigma[np.ix_(idx, idx)]
            o = np.ones(len(idx))
            m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
            b = np.concatenate([m, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b)
                w_sub = sol[:len(idx)]
            except np.linalg.LinAlgError:
                w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n)
            w[idx] = w_sub
            return w

    best_w = np.full(n, 1.0/n)
    best_sr = -1e9
    lambdas = np.logspace(-3, 3, 31)
    for lmbd in lambdas:
        active = np.ones(n, dtype=bool)
        w = None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            if not (w < 0).any():
                break
            worst = np.argmin(w)
            active[worst] = False
        if w is None:
            continue
        w = np.clip(w, 0, None)
        if w.sum() <= 0:
            continue
        w = w / w.sum()
        mu_p = float(mu @ w)
        vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8:
            continue
        sr = mu_p / vol_p
        if sr > best_sr:
            best_sr = sr
            best_w = w.copy()
    return best_w

def aggregate_and_apply(all_trades: pd.DataFrame, data_map: Dict[str, pd.DataFrame], bench_df: pd.DataFrame, cfg: Config):
    """Size, filter (52w optional), deploy-cap, MVO; build equity & roundtrips; compute metrics."""
    if all_trades.empty:
        return all_trades, pd.Series(dtype=float), {}

    side_order = {"BUY": 0, "SELL": 1}
    all_trades = (all_trades
        .assign(_sorder=all_trades["side"].map(side_order))
        .sort_values(by=["date", "_sorder"], kind="stable")
        .drop(columns=["_sorder"])
        .reset_index(drop=True)
    )
    all_trades["date"] = pd.to_datetime(all_trades["date"])

    equity_curve = []
    dates = sorted(all_trades["date"].unique().tolist())
    cash = cfg.initial_capital
    open_positions = {}
    completed_legs = []

    global APPLY_FEES
    APPLY_FEES = cfg.apply_fees

    def _get_close_on(tkr, dt):
        df = data_map.get(tkr)
        if df is None or df.empty:
            return np.nan
        if dt in df.index:
            return float(df.loc[dt, "Close"])
        prev = df[df.index <= dt]
        if prev.empty:
            return np.nan
        return float(prev["Close"].iloc[-1])

    if dates:
        seed_date = pd.to_datetime(dates[0]) - pd.Timedelta(days=1)
        equity_curve.append((seed_date, float(cash)))

    for dt in dates:
        day_trades = all_trades[all_trades["date"] == dt].copy()

        # ---- SELL first ----
        for _, tr in day_trades[day_trades["side"] == "SELL"].iterrows():
            tkr = tr["ticker"]
            price = float(tr["price"])
            pos = open_positions.get(tkr)
            if pos is None:
                continue
            shares = int(pos["shares"])
            turnover_sell = shares * price
            fee = calc_fees(0.0, turnover_sell)
            pnl = (price - pos["entry_px"]) * shares
            cash += (turnover_sell - fee)
            realized = pnl - fee - pos.get("buy_fee", 0.0)
            completed_legs.append({
                "ticker": tkr, "side": "SELL", "date": dt, "price": price,
                "shares": shares, "reason": tr.get("reason",""),
                "turnover": turnover_sell, "fees_inr": fee, "pnl_inr": realized,
                "rsi": tr.get("rsi", np.nan),
                "macd": tr.get("macd", np.nan), "macd_sig": tr.get("macd_sig", np.nan),
                "bb_up": tr.get("bb_up", np.nan), "bb_lo": tr.get("bb_lo", np.nan),
                "ema_fast": tr.get("ema_fast", np.nan), "ema_slow": tr.get("ema_slow", np.nan), "ema_htf": tr.get("ema_htf", np.nan),
                "trend_strength": tr.get("trend_strength", np.nan), "close": tr.get("close", np.nan), "high_52w": tr.get("high_52w", np.nan),
                "volar": tr.get("volar", np.nan), "mvo_weight": np.nan, "alloc_inr": np.nan, "adx": tr.get("adx", np.nan)
            })
            log.info("Exit %-12s px=%8.2f shares=%6d reason=%s net=%.2f cash=%.2f",
                     tkr, price, shares, tr.get("reason",""), realized, cash)
            del open_positions[tkr]

        # ---- BUY candidates today ----
        buys_today = day_trades[day_trades["side"] == "BUY"].copy()

        # Optional 52w filter
        if cfg.enforce_52w_filter and not buys_today.empty:
            keep = []
            for _, rr in buys_today.iterrows():
                df_t = data_map.get(rr["ticker"])
                if df_t is None or df_t.empty or dt not in df_t.index:
                    continue
                close = float(df_t.loc[dt, "Close"])
                hist = df_t["Close"].loc[:dt]
                window = hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
                high_52w = float(window.max())
                if high_52w > 0 and close >= cfg.within_pct_of_52w_high * high_52w:
                    keep.append(rr)
            buys_today = pd.DataFrame(keep) if keep else pd.DataFrame(columns=buys_today.columns)

        # Exclude already-held tickers
        if not buys_today.empty:
            buys_today = buys_today[~buys_today["ticker"].isin(open_positions.keys())]

        # VOLAR ranking
        if not buys_today.empty:
            tickers = buys_today["ticker"].tolist()
            volar_scores = compute_volar_scores(dt, tickers, data_map, bench_df, cfg.volar_lookback)
            buys_today["volar"] = buys_today["ticker"].map(volar_scores)
            buys_today = buys_today.sort_values("volar", ascending=False).reset_index(drop=True)

        slots = cfg.max_concurrent_positions - len(open_positions)
        selected = pd.DataFrame(columns=buys_today.columns)
        if slots > 0 and not buys_today.empty:
            selected = buys_today.head(min(cfg.top_k_daily, slots)).copy()

        if not selected.empty:
            log.info("Selected %d BUY candidates on %s:", selected.shape[0], dt.date())
            for i, rr in selected.reset_index(drop=True).iterrows():
                log.info("  %-12s volar=%6.2f rank=%d px=%8.2f", rr["ticker"], rr.get("volar",0.0), i+1, rr["price"])

            # MVO sizing on trailing returns up to dt
            names = selected["ticker"].tolist()
            rets = []
            for t in names:
                df_t = data_map.get(t)
                ser = df_t["Close"].loc[:dt].pct_change().dropna().iloc[-cfg.volar_lookback:]
                rets.append(ser)
            R = pd.concat(rets, axis=1)
            R.columns = names
            R = R.dropna()
            if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)) or R.shape[1] == 0:
                weights = np.full(len(names), 1.0/len(names))
            else:
                mu = R.mean().values
                Sigma = R.cov().values
                weights = markowitz_long_only(mu, Sigma)

            # Cap daily deployed cash
            deploy_cash = max(0.0, float(cash)) * float(cfg.deploy_cash_frac)

            if deploy_cash <= 0:
                log.info("No deployable cash (cap=%.0f%%) on %s", 100*cfg.deploy_cash_frac, dt.date())
            else:
                alloc = (weights / weights.sum()) * deploy_cash if weights.sum()>0 else np.full(len(names), deploy_cash/len(names))
                rank_map = {row["ticker"]: (idx+1) for idx, (_, row) in enumerate(selected.iterrows())}
                for w_amt, t in zip(alloc, names):
                    df_t = data_map[t]
                    price = float(df_t.loc[dt, "Close"] if dt in df_t.index else df_t["Close"].loc[:dt].iloc[-1])
                    shares = int(math.floor(w_amt / price))
                    if shares <= 0:
                        log.info("Skip BUY %-12s (alloc %.2f too small)", t, w_amt)
                        continue
                    turn = shares * price
                    fee = calc_fees(turn, 0.0)
                    total_cost = turn + fee
                    if total_cost > cash:
                        shares = int(math.floor((cash - fee) / price))
                        if shares <= 0:
                            log.info("Skip BUY %-12s due to cash/fees", t)
                            continue
                        turn = shares * price
                        total_cost = turn + fee
                    cash -= total_cost
                    open_positions[t] = {"entry_date": dt, "entry_px": price, "shares": shares, "buy_fee": fee, "entry_reason": "entry"}
                    row_sel = selected[selected["ticker"]==t].iloc[0]
                    volar_val = float(row_sel.get("volar", np.nan))
                    rank_pos = rank_map.get(t, np.nan)
                    high_52w = float(row_sel.get("high_52w", np.nan))
                    close_val = float(row_sel.get("close", np.nan))
                    pct_52w = (close_val / high_52w) if (high_52w and high_52w>0) else np.nan
                    mvo_weight_today = (w_amt / deploy_cash) if deploy_cash > 0 else 0.0
                    sig_reason = row_sel.get("signal_reason", "")
                    reason_text = (
                        f"{sig_reason}; "
                        f"52w%={pct_52w:.1%}"
                        f"{' (>= ' + f'{cfg.within_pct_of_52w_high:.0%}' + ')' if cfg.enforce_52w_filter else ''}; "
                        f"VOLAR rank {int(rank_pos)}/{len(names)} (VOLAR={volar_val:.2f}); "
                        f"MVO weight={mvo_weight_today:.1%} of capped cash ({100*cfg.deploy_cash_frac:.0f}% of available)"
                    )
                    completed_legs.append({
                        "ticker": t, "side": "BUY", "date": dt, "price": price,
                        "shares": shares, "reason": reason_text,
                        "turnover": turn, "fees_inr": fee, "pnl_inr": 0.0,
                        "rsi": row_sel.get("rsi", np.nan),
                        "macd": row_sel.get("macd", np.nan), "macd_sig": row_sel.get("macd_sig", np.nan),
                        "bb_up": row_sel.get("bb_up", np.nan), "bb_lo": row_sel.get("bb_lo", np.nan),
                        "ema_fast": row_sel.get("ema_fast", np.nan), "ema_slow": row_sel.get("ema_slow", np.nan), "ema_htf": row_sel.get("ema_htf", np.nan),
                        "trend_strength": row_sel.get("trend_strength", np.nan), "close": close_val, "high_52w": high_52w,
                        "volar": volar_val, "mvo_weight": float(mvo_weight_today), "alloc_inr": float(w_amt), "adx": row_sel.get("adx", np.nan)
                    })
                    log.info("BUY %-12s px=%8.2f sh=%6d fee=%.2f cash=%.2f :: %s",
                             t, price, shares, fee, cash, reason_text)

        # MTM valuation
        mtm = 0.0
        for _tkr, pos in open_positions.items():
            px = _get_close_on(_tkr, dt)
            if not np.isnan(px):
                mtm += pos["shares"] * px
        total_equity = cash + mtm
        equity_curve.append((dt, float(total_equity)))

    eq_ser = pd.Series([e for _, e in equity_curve], index=[d for d, _ in equity_curve])
    legs_df = pd.DataFrame(completed_legs).sort_values(["date", "ticker", "side"]).reset_index(drop=True)

    # Build roundtrips
    roundtrips = []
    by_tkr_open = {}
    for _, leg in legs_df.iterrows():
        tkr = leg["ticker"]
        if leg["side"] == "BUY":
            by_tkr_open[tkr] = leg
        else:
            buy = by_tkr_open.pop(tkr, None)
            if buy is None:
                continue
            fees_total = float(buy.get("fees_inr", 0.0) + leg.get("fees_inr", 0.0))
            gross_pnl = (leg["price"] - buy["price"]) * buy["shares"]
            net_pnl   = gross_pnl - fees_total
            ret_pct   = (leg["price"] / buy["price"] - 1.0) * 100.0
            days_held = (pd.to_datetime(leg["date"]) - pd.to_datetime(buy["date"])).days
            roundtrips.append({
                "ticker": tkr,
                "entry_date": pd.to_datetime(buy["date"]),
                "entry_price": float(buy["price"]),
                "exit_date": pd.to_datetime(leg["date"]),
                "exit_price": float(leg["price"]),
                "days_held": int(days_held),
                "shares": int(buy["shares"]),
                "entry_reason": buy.get("reason",""),
                "exit_reason": leg.get("reason",""),
                "gross_pnl_inr": float(gross_pnl),
                "fees_total_inr": float(fees_total),
                "net_pnl_inr": float(net_pnl),
                "return_pct": float(ret_pct),
                # snapshot at entry
                "rsi_entry": float(buy.get("rsi", np.nan)),
                "macd_entry": float(buy.get("macd", np.nan)), "macd_sig_entry": float(buy.get("macd_sig", np.nan)),
                "bb_up_entry": float(buy.get("bb_up", np.nan)), "bb_lo_entry": float(buy.get("bb_lo", np.nan)),
                "ema_fast_entry": float(buy.get("ema_fast", np.nan)),
                "ema_slow_entry": float(buy.get("ema_slow", np.nan)),
                "ema_htf_entry": float(buy.get("ema_htf", np.nan)),
                "trend_strength_entry": float(buy.get("trend_strength", np.nan)),
                "close_entry": float(buy.get("close", np.nan)),
                "high_52w_entry": float(buy.get("high_52w", np.nan)),
                "volar_entry": float(buy.get("volar", np.nan)),
                "mvo_weight_entry": float(buy.get("mvo_weight", np.nan)),
                "alloc_inr_entry": float(buy.get("alloc_inr", np.nan)),
                "adx_entry": float(buy.get("adx", np.nan)),
            })
    trips_df = pd.DataFrame(roundtrips).sort_values(["entry_date","ticker"]).reset_index(drop=True)

    metrics = compute_metrics(eq_ser, legs_df)
    return legs_df, trips_df, eq_ser, metrics

def compute_metrics(equity: pd.Series, legs_df: pd.DataFrame):
    out = {}
    if equity is None or equity.empty:
        return out
    eq = equity.dropna()
    daily_ret = eq.pct_change().fillna(0.0)

    days = (eq.index[-1] - eq.index[0]).days or 1
    years = days / 365.25
    cagr = (eq.iloc[-1] / eq.iloc[0]) ** (1/years) - 1 if years > 0 else 0.0

    sharpe = (daily_ret.mean() / daily_ret.std(ddof=0) * np.sqrt(252)) if daily_ret.std(ddof=0) > 0 else 0.0

    cummax = eq.cummax()
    dd = (eq - cummax) / cummax
    max_dd = dd.min()

    wins = 0
    n_sells = legs_df[legs_df["side"] == "SELL"].shape[0] if legs_df is not None and not legs_df.empty else 0
    for _, r in legs_df[legs_df["side"] == "SELL"].iterrows():
        if float(r.get("pnl_inr", 0.0)) > 0:
            wins += 1
    win_rate = (wins / n_sells) * 100.0 if n_sells > 0 else 0.0

    out.update({
        "start_equity_inr": float(eq.iloc[0]),
        "final_equity_inr": float(eq.iloc[-1]),
        "cagr_pct": float(cagr * 100),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd * 100),
        "win_rate_pct": float(win_rate),
        "n_trades": int(n_sells),
    })
    return out

def plot_equity(equity: pd.Series, out_path: str):
    if equity is None or equity.empty:
        return
    try:
        plt.figure(figsize=(10,5))
        plt.plot(equity.index, equity.values)
        plt.title("Equity Curve")
        plt.xlabel("Date")
        plt.ylabel("Equity (INR)")
        plt.tight_layout()
        plt.savefig(out_path)
        plt.close()
    except Exception:
        pass

def backtest(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    log.info("Universe: loading static symbols...")
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Loaded %d symbols.", len(symbols))

    log.info("Data: fetching OHLCV from yfinance (adjusted)...")
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Downloaded %d symbols with data.", len(data_map))

    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Benchmark selected: %s", bench_tkr)

    log.info("Signals: generating indicators and candidate entries...")
    all_trades = []
    for i, tkr in enumerate(symbols, 1):
        df = data_map.get(tkr)
        if df is None or df.empty:
            continue
        tr, _ = simulate_ticker(tkr, df, cfg)
        if not tr.empty:
            all_trades.append(tr)
        if i % 50 == 0:
            log.info("  processed %d/%d tickers...", i, len(symbols))

    if not all_trades:
        log.warning("No signals generated; loosen thresholds or turn off filters.")
        return None, None, None, {}
    all_trades = pd.concat(all_trades, ignore_index=True)

    log.info("Portfolio: cap daily deploy to %.0f%%; 52w%s; top-%d by VOLAᵣ; MVO; max %d positions.",
             cfg.deploy_cash_frac*100,
             " ENFORCED" if cfg.enforce_52w_filter else " OFF",
             cfg.top_k_daily, cfg.max_concurrent_positions)

    legs_df, trips_df, equity, metrics = aggregate_and_apply(all_trades, data_map, bench_df, cfg)

    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    legs_path = os.path.join(cfg.out_dir, f"trades_legs_{stamp}.csv")
    trips_path = os.path.join(cfg.out_dir, f"trades_roundtrips_{stamp}.csv")
    equity_path = os.path.join(cfg.out_dir, f"equity_{stamp}.csv")
    metrics_path = os.path.join(cfg.out_dir, f"metrics_{stamp}.json")
    eq_plot_path = os.path.join(cfg.out_dir, f"equity_{stamp}.png")

    if legs_df is not None:
        legs_df.to_csv(legs_path, index=False)
    if trips_df is not None:
        trips_df.to_csv(trips_path, index=False)
    if equity is not None:
        pd.DataFrame({"date": equity.index, "equity": equity.values}).to_csv(equity_path, index=False)
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)

    if cfg.plot and equity is not None:
        plot_equity(equity, eq_plot_path)

    log.info("=== METRICS ===\n%s", json.dumps(metrics, indent=2))
    log.info("Files written:\n  %s\n  %s\n  %s\n  %s", legs_path, trips_path, equity_path, metrics_path)
    if cfg.plot:
        log.info("  %s", eq_plot_path)

def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)

    # Use your big NIFTY-500 style list from the previous script
    CFG.static_symbols = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']

    # Or: CFG.static_symbols_path = "nifty500.txt"
    backtest(CFG)

if __name__ == "__main__":
    main()


2025-10-08 00:14:16 | INFO | Universe: loading static symbols...
2025-10-08 00:14:16 | INFO | Loaded 500 symbols.
2025-10-08 00:14:16 | INFO | Data: fetching OHLCV from yfinance (adjusted)...
2025-10-08 00:17:18 | INFO | Downloaded 500 symbols with data.
2025-10-08 00:17:19 | ERROR | HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ^CNX500"}}}
2025-10-08 00:17:20 | ERROR | 
1 Failed download:
2025-10-08 00:17:20 | ERROR | ['^CNX500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-08 00:17:20 | INFO | Using benchmark: ^CRSLDX
2025-10-08 00:17:20 | INFO | Benchmark selected: ^CRSLDX
2025-10-08 00:17:20 | INFO | Signals: generating indicators and candidate entries...
2025-10-08 00:17:25 | INFO |   processed 50/500 tickers...
2025-10-08 00:17:32 | INFO |   processed 100/500 tickers...
2025-10-08 00:17:38 | INFO |   processed 150/500 tickers...
2025-10-08 00:17:45 | INFO |   processed 200/500 tickers...
